In [341]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from py_session import py_session
import time
try:
  # Use the %tensorflow_version magic if in colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import numpy as np
import collections
from sklearn.utils import shuffle
from tensorflow_federated import python as tff


In [169]:
le= LabelEncoder()

In [ ]:
datasetScenearioA120 = pd.read_csv("datasetScenearioA120.csv", header=None, 
 names=['duration',
 'total_fiat',
 'total_biat',
 'min_fiat',
 'min_biat',
 'max_fiat',
 'max_biat',
 'mean_fiat',
 'mean_biat',
 'flowPktsPerSecond',
 'flowBytesPerSecond',
 'min_flowiat',
 'max_flowiat',
 'mean_flowiat',
 'std_flowiat',
 'min_active',
 'mean_active',
 'max_active',
 'std_active',
 'min_idle',
 'mean_idle',
 'max_idle',
 'std_idle',
 'class'])

datasetScenearioA120['encoded_class']=le.fit_transform(datasetScenearioA120['class'])

In [14]:
le.classes_

array(['Non-VPN', 'VPN'], dtype=object)

In [15]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'Non-VPN': 0, 'VPN': 1}


In [ ]:
datasetScenearioA120=datasetScenearioA120.drop(columns=['class'])

datasetA120Norm=((datasetScenearioA120-datasetScenearioA120.mean())/(datasetScenearioA120.std()))

datasetA120Norm['encoded_class']=datasetScenearioA120['encoded_class']

In [ ]:
datasetScenearioA60 = pd.read_csv("datasetScenearioA60.csv", header=None, 
 names=['duration',
 'total_fiat',
 'total_biat',
 'min_fiat',
 'min_biat',
 'max_fiat',
 'max_biat',
 'mean_fiat',
 'mean_biat',
 'flowPktsPerSecond',
 'flowBytesPerSecond',
 'min_flowiat',
 'max_flowiat',
 'mean_flowiat',
 'std_flowiat',
 'min_active',
 'mean_active',
 'max_active',
 'std_active',
 'min_idle',
 'mean_idle',
 'max_idle',
 'std_idle',
 'class'])

datasetScenearioA60['encoded_class']=le.fit_transform(datasetScenearioA60['class'])

datasetScenearioA60=datasetScenearioA60.drop(columns=['class'])


datasetA60Norm=((datasetScenearioA60-datasetScenearioA60.mean())/(datasetScenearioA60.std()))

datasetA60Norm['encoded_class']=datasetScenearioA60['encoded_class']

In [ ]:
datasetScenearioA30 = pd.read_csv("datasetScenearioA30.csv", header=None, 
 names=['duration',
 'total_fiat',
 'total_biat',
 'min_fiat',
 'min_biat',
 'max_fiat',
 'max_biat',
 'mean_fiat',
 'mean_biat',
 'flowPktsPerSecond',
 'flowBytesPerSecond',
 'min_flowiat',
 'max_flowiat',
 'mean_flowiat',
 'std_flowiat',
 'min_active',
 'mean_active',
 'max_active',
 'std_active',
 'min_idle',
 'mean_idle',
 'max_idle',
 'std_idle',
 'class'])

datasetScenearioA30['encoded_class']=le.fit_transform(datasetScenearioA30['class'])

datasetScenearioA30=datasetScenearioA30.drop(columns=['class'])

datasetA30Norm=((datasetScenearioA30-datasetScenearioA30.mean())/(datasetScenearioA30.std()))

datasetA30Norm['encoded_class']=datasetScenearioA30['encoded_class']



In [ ]:
datasetScenearioA15 = pd.read_csv("datasetScenearioA15.csv", header=None, 
 names=['duration',
 'total_fiat',
 'total_biat',
 'min_fiat',
 'min_biat',
 'max_fiat',
 'max_biat',
 'mean_fiat',
 'mean_biat',
 'flowPktsPerSecond',
 'flowBytesPerSecond',
 'min_flowiat',
 'max_flowiat',
 'mean_flowiat',
 'std_flowiat',
 'min_active',
 'mean_active',
 'max_active',
 'std_active',
 'min_idle',
 'mean_idle',
 'max_idle',
 'std_idle',
 'class'])datasetScenearioA15['encoded_class']=le.fit_transform(datasetScenearioA15['class'])

datasetScenearioA15=datasetScenearioA15.drop(columns=['class'])

datasetA15Norm=((datasetScenearioA15-datasetScenearioA15.mean())/(datasetScenearioA15.std()))

datasetA15Norm['encoded_class']=datasetScenearioA15['encoded_class']

In [ ]:
dataNorm=datasetA120Norm

dataNorm=dataNorm.append(datasetA60Norm, ignore_index=True)
dataNorm=dataNorm.append(datasetA30Norm, ignore_index=True)
dataNorm=dataNorm.append(datasetA15Norm, ignore_index=True)

In [55]:
dataNorm.dtypes

duration              float64
total_fiat            float64
total_biat            float64
min_fiat              float64
min_biat              float64
max_fiat              float64
max_biat              float64
mean_fiat             float64
mean_biat             float64
flowPktsPerSecond     float64
flowBytesPerSecond    float64
min_flowiat           float64
max_flowiat           float64
mean_flowiat          float64
std_flowiat           float64
min_active            float64
mean_active           float64
max_active            float64
std_active            float64
min_idle              float64
mean_idle             float64
max_idle              float64
std_idle              float64
encoded_class           int64
dtype: object

In [56]:
dataNorm

,duration,total_fiat,total_biat,min_fiat,min_biat,max_fiat,max_biat,mean_fiat,mean_biat,flowPktsPerSecond,flowBytesPerSecond,min_flowiat,max_flowiat,mean_flowiat,std_flowiat,min_active,mean_active,max_active,std_active,min_idle,mean_idle,max_idle,std_idle,encoded_class
0,1.387643,-0.311477,-0.276861,0.628009,0.823722,-0.411613,-0.392982,-0.162273,-0.155449,0.649243,-0.084152,-0.105118,0.543530,-0.446548,-0.355567,-0.378909,-0.325365,0.526630,0.521968,-0.381806,-0.351999,0.545133,0.554642,0
1,1.433055,-0.311477,-0.276861,0.628704,0.818574,-0.409091,-0.394245,-0.155295,-0.162247,0.607625,-0.084126,-0.105118,0.539895,-0.445457,-0.354658,-0.376179,-0.345179,0.524956,0.468308,-0.382291,-0.360140,0.541502,0.495028,0
2,0.876661,-0.311476,-0.276861,0.261788,0.379124,-0.376589,-0.349008,-0.042972,0.009202,0.199013,-0.084211,-0.105117,0.198530,-0.400055,-0.263476,-0.331239,-0.176311,0.190405,0.661566,-0.374314,-0.195001,0.200576,0.753111,0
3,0.396602,-0.311477,-0.276861,0.320534,0.449830,-0.396896,-0.378565,-0.100082,-0.083194,0.187662,-0.084166,-0.105117,0.253455,-0.429132,-0.312051,-0.371975,-0.201415,0.239987,0.728475,-0.377779,-0.223393,0.255431,0.760556,0
4,1.441082,1.925923,2.154681,0.452196,0.613841,1.628258,1.811737,-0.381014,-0.464994,-0.058262,-0.084269,-0.045857,0.379062,2.091895,0.944604,0.600347,0.496862,0.359364,-0.368088,0.610032,0.508947,0.380876,-0.358025,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59701,-0.675617,-0.271345,-0.269416,-0.312038,-0.296472,-0.319534,-0.369323,-0.194815,-0.243745,-0.107057,-0.050335,0.071252,-0.272730,-0.287360,-0.292869,-0.242290,-0.268084,-0.292185,-0.178574,-0.223616,-0.246777,-0.268493,-0.162674,0
59702,-0.677088,-0.271345,-0.269416,-0.312038,-0.296472,-0.319534,-0.369323,-0.194815,-0.243745,-0.106477,-0.050055,0.023528,-0.274255,-0.302425,-0.292869,-0.242290,-0.268084,-0.292185,-0.178574,-0.223616,-0.246777,-0.268493,-0.162674,0
59703,-0.675986,-0.271345,-0.269416,-0.312038,-0.296472,-0.319534,-0.369323,-0.194815,-0.243745,-0.106945,-0.050281,0.059259,-0.273113,-0.291146,-0.292869,-0.242290,-0.268084,-0.292185,-0.178574,-0.223616,-0.246777,-0.268493,-0.162674,0
59704,-0.658908,-0.271345,-0.269416,-0.312038,-0.296472,-0.319534,-0.369323,-0.194815,-0.243745,-0.108152,-0.050865,0.613413,-0.255402,-0.116224,-0.292869,-0.242290,-0.268084,-0.292185,-0.178574,-0.223616,-0.246777,-0.268493,-0.162674,0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_total_numeric_features=dataNorm[['duration',
 'total_fiat',
 'total_biat',
 'min_fiat',
 'min_biat',
 'max_fiat',
 'max_biat',
 'mean_fiat',
 'mean_biat',
 'flowPktsPerSecond',
 'flowBytesPerSecond',
 'min_flowiat',
 'max_flowiat',
 'mean_flowiat',
 'std_flowiat',
 'min_active',
 'mean_active',
 'max_active',
 'std_active',
 'min_idle',
 'mean_idle',
 'max_idle',
 'std_idle']].to_numpy()

In [ ]:
df_total_numeric_label=dataNorm[['encoded_class']].to_numpy()

In [ ]:
ohe_label = to_categorical(df_total_numeric_label)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_total_numeric_features, ohe_label,
                                                    stratify=ohe_label, 
                                                    test_size=0.2)

In [ ]:
import numpy as np
(unique, counts) = np.unique(y_train, return_counts=True,axis=0)

In [ ]:
unique

array([[0., 1.],
       [1., 0.]], dtype=float32)

In [70]:
total_test=len(X_test)
total_test

11942

In [71]:
total_train=len(X_train)
total_train

47764

In [ ]:
y_train=y_train.astype('int32')

In [ ]:
x_train=x_train.astype('float32')

In [ ]:
x_train[1].shape

(23,)

In [ ]:
x_train[1]

array([ 0.7010088 , -0.27985027, -0.27788177,  0.96749234,  1.1173689 ,
       -0.28698382, -0.26231286,  0.15936655,  0.43103838, -0.13398038,
       -0.05413245, -0.08564138,  0.70858586, -0.275996  , -0.03335306,
       -0.23655762,  0.085145  ,  0.67940474,  1.6422577 , -0.23330466,
        0.07355642,  0.7129298 ,  1.7827014 ], dtype=float32)

In [ ]:
y_test=y_test.astype('int32')

In [ ]:
x_test=x_test.astype('float32')

In [ ]:
CLIENTS=2
EPOCHS=5
BATCH_SIZE_train = int(total_train/2) # 23882
BATCH_SIZE_test = int(total_test/2) #5971
BATCH_SIZE = 100

In [362]:
def generate_clients_datasets(n, BATCH_SIZE_EXT, source_x, source_y):
    clients_dataset=[]
    for i in range(n):
        dataset=tf.data.Dataset.from_tensor_slices((source_x[i*BATCH_SIZE_EXT:(i+1)*BATCH_SIZE_EXT], source_y[i*BATCH_SIZE_EXT:(i+1)*BATCH_SIZE_EXT]))
        dataset = dataset.batch(BATCH_SIZE)
        clients_dataset.append(dataset)
    return clients_dataset

In [363]:
train_dataset_central=generate_clients_datasets(1, total_train*2, x_train, y_train)

In [364]:
test_dataset_central=generate_clients_datasets(1, total_test*2, x_test, y_test)

In [365]:
train_dataset_individual=generate_clients_datasets(CLIENTS, BATCH_SIZE_train, x_train, y_train)

In [366]:
test_dataset_individual=generate_clients_datasets(CLIENTS, BATCH_SIZE_test, x_test, y_test)

In [367]:
batch_of_samples = tf.nest.map_structure(
    lambda x: x.numpy(), iter(train_dataset_individual[0]).next()
)

In [ ]:
Epochs = 600

In [ ]:
display_epoch=Epochs
t = np.arange(1, display_epoch+1, 1)


In [369]:
federated_model_1 = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(23,)), 

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),

    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(14, activation='softmax')
      
    ])

In [370]:
federated_model_1.load_weights('ScenarioB-weights.hdf5')

In [372]:
model_2 = tf.keras.models.Sequential([
                                           
    tf.keras.layers.Input(shape=(23,)), 

    tf.keras.layers.Dense(512, activation='relu',trainable=False),
    tf.keras.layers.Dense(512, activation='relu',trainable=False),
    tf.keras.layers.Dense(512, activation='relu',trainable=False),

    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(512, activation='relu',trainable=False),
    tf.keras.layers.Dense(512, activation='relu',trainable=False),
    tf.keras.layers.Dense(512, activation='relu',trainable=False),
    
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')

])



In [374]:
for i in [0,1,2,4,5,6]:
    model_2.layers[i].set_weights(federated_model_1.layers[i].get_weights())

In [375]:
def model_instance_2():
    
    federated_model_2 = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(23,)), 

    tf.keras.layers.Dense(512, activation='relu',trainable=False),
    tf.keras.layers.Dense(512, activation='relu',trainable=False),
    tf.keras.layers.Dense(512, activation='relu',trainable=False),

    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(512, activation='relu',trainable=False),
    tf.keras.layers.Dense(512, activation='relu',trainable=False),
    tf.keras.layers.Dense(512, activation='relu',trainable=False),
    
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
      
    ])
    
    federated_model_2.compile(optimizer='sgd', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])

    
    return tff.learning.from_compiled_keras_model(federated_model_2, batch_of_samples)
    

In [265]:
model_2.trainable_weights

[<tf.Variable 'dense_156/kernel:0' shape=(512, 512) dtype=float32, numpy=
 array([[ 0.00272153, -0.0473405 ,  0.0003283 , ...,  0.02654294,
         -0.03029433,  0.01658034],
        [ 0.06301145,  0.04210535, -0.04652577, ..., -0.00526372,
         -0.01437982,  0.05336682],
        [-0.01953186, -0.04796439, -0.04199747, ..., -0.04948372,
          0.07162526,  0.04433641],
        ...,
        [ 0.01054312, -0.02929157, -0.03405128, ...,  0.01560223,
         -0.03365923, -0.00716131],
        [-0.00753363, -0.00220622,  0.02195337, ...,  0.06644117,
         -0.00250603,  0.01114522],
        [-0.05843182, -0.01743961,  0.00874876, ...,  0.02733467,
         -0.0355105 , -0.04457568]], dtype=float32)>,
 <tf.Variable 'dense_156/bias:0' shape=(512,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [266]:
model_2.non_trainable_weights

[<tf.Variable 'dense_150/kernel:0' shape=(23, 512) dtype=float32, numpy=
 array([[-0.02366572, -0.23189053,  0.09492652, ...,  0.13089958,
          0.07558083,  0.07642329],
        [-0.07560708,  0.19337669, -0.01664433, ..., -0.15662646,
         -0.22840007, -0.08208055],
        [ 0.08279371, -0.03251741,  0.15696311, ...,  0.18394352,
         -0.06625906, -0.1606164 ],
        ...,
        [-0.14959942, -0.11298791,  0.07859192, ..., -0.17828497,
          0.03424055, -0.06155569],
        [-0.15046073,  0.05576283,  0.07601173, ..., -0.07278212,
          0.1037025 , -0.10117775],
        [ 0.18278578, -0.00103905,  0.16819724, ..., -0.01271042,
         -0.29987824, -0.21689793]], dtype=float32)>,
 <tf.Variable 'dense_150/bias:0' shape=(512,) dtype=float32, numpy=
 array([-5.07850535e-02, -8.92703421e-03, -5.81309870e-02, -1.04120769e-01,
         9.31708701e-03,  5.07602748e-03, -5.19848466e-01, -4.58212271e-02,
        -6.23376779e-02, -2.40677558e-02, -5.87757975e-02, -1.67

In [376]:
def model_instance_2_baseline():

    fedrated_model_2_baseline = tf.keras.models.Sequential([
                                           
    tf.keras.layers.Input(shape=(23,)), 

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),

    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')

    ])

    fedrated_model_2_baseline.compile(optimizer='sgd', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])

    return tff.learning.from_compiled_keras_model(fedrated_model_2_baseline, batch_of_samples)
    

In [377]:
federated_data=train_dataset_individual

In [ ]:
federated_learning_iterative_process_2 = tff.learning.build_federated_averaging_process(
    model_instance_2 )
federated_learning_iterative_process_baseline = tff.learning.build_federated_averaging_process(
    model_instance_2_baseline )



In [ ]:
state_2 = federated_learning_iterative_process_2.initialize()

In [380]:
state_2 = tff.learning.state_with_new_model_weights(
    state_2,
    trainable_weights=[v.numpy() for v in model_2.trainable_weights],
    non_trainable_weights=[
        v.numpy() for v in model_2.non_trainable_weights
    ])

In [382]:

state_2_baseline = federated_learning_iterative_process_baseline.initialize()

In [ ]:
evaluation_2 = tff.learning.build_federated_evaluation(model_instance_2)
evaluation_2_baseline = tff.learning.build_federated_evaluation(model_instance_2_baseline)


In [ ]:
filepath_2="ScenarioA-weights-best-2.hdf5"
filepath_2_baseline="ScenarioA-weights-best-2-baseline.hdf5"

In [274]:
federated_model_val = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(23,)), 

    tf.keras.layers.Dense(512, activation='relu',trainable=False),
    tf.keras.layers.Dense(512, activation='relu',trainable=False),
    tf.keras.layers.Dense(512, activation='relu',trainable=False),

    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(512, activation='relu',trainable=False),
    tf.keras.layers.Dense(512, activation='relu',trainable=False),
    tf.keras.layers.Dense(512, activation='relu',trainable=False),
    
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
      
    ])

In [275]:
federated_model_val_baseline = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(23,)), 

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),

    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
      
    ])

In [303]:
EPOCHS = 600

In [ ]:
fed_sparse_categorical_accuracy=[]
time_2=[]
fed_loss=[]
fed_val_sparse_categorical_accuracy=[]
fed_val_loss=[]
fed_sparse_categorical_accuracy_baseline=[]
fed_loss_baseline=[]
fed_val_sparse_categorical_accuracy_baseline=[]
fed_val_loss_baseline=[]
time_baseline=[]
for n in range(EPOCHS):
    start = time.time()
    state_2, metrics_2 = federated_learning_iterative_process_2.next(state_2, [test_dataset_central[0],test_dataset_central[0]])
    end = time.time()
    time_2.append(end - start)
    start = time.time()
    state_2_baseline, metrics_2_baseline = federated_learning_iterative_process_baseline.next(state_2_baseline, [test_dataset_central[0],test_dataset_central[0]])
    end = time.time()
    time_baseline.append(end - start)
    fed_sparse_categorical_accuracy.append(metrics_2[0])
    fed_loss.append(metrics_2[1])
    test_metrics = evaluation_2(state_2.model, test_dataset_individual)
    fed_val_sparse_categorical_accuracy.append(test_metrics[0])
    fed_val_loss.append(test_metrics[1])
    fed_sparse_categorical_accuracy_baseline.append(metrics_2_baseline[0])
    fed_loss_baseline.append(metrics_2_baseline[1])
    test_metrics_baseline = evaluation_2_baseline(state_2_baseline.model, test_dataset_individual)
    fed_val_sparse_categorical_accuracy_baseline.append(test_metrics_baseline[0])
    fed_val_loss_baseline.append(test_metrics_baseline[1])
    if fed_val_sparse_categorical_accuracy [n] > fed_val_sparse_categorical_accuracy [n-1]:
        tffweights = tff.learning.model_utils.ModelWeights.from_tff_result(state_2.model)
        tffweights.assign_weights_to(federated_model_val)
        federated_model_val.save_weights(filepath_2)
    if fed_val_sparse_categorical_accuracy_baseline [n] > fed_val_sparse_categorical_accuracy_baseline [n-1]:
        tffweights = tff.learning.model_utils.ModelWeights.from_tff_result(state_2_baseline.model)
        tffweights.assign_weights_to(federated_model_val_baseline)
        federated_model_val.save_weights(filepath_2_baseline)
        
    print('model 2 - round  {}, fed_loss: {} - fed_sparse_categorical_accuracy: {} - fed_val_loss: {}- fed_val_sparse_categorical_accuracy: {}'.format(n+1, metrics_2[1], metrics_2[0], test_metrics[1], test_metrics[0] ))
    print('model 2 baseline - round  {}, fed_loss: {} - fed_sparse_categorical_accuracy: {} - fed_val_loss: {}- fed_val_sparse_categorical_accuracy: {}'.format(n+1, metrics_2_baseline[1], metrics_2_baseline[0], test_metrics_baseline[1], test_metrics_baseline[0] ))


<p style="text-align:center">
    <img src="target.png" width="600">
</p>
